# Country Club Business Analysis with SQL
## Jack Wolfgramm

Hi, welcome to the business analysis using SQL notebook! The point here is to answer some basic business questions and do some
exploratory data analysis using SQL, just like a real data scientist would. I have much more practice writing SQL queries than
what I can put in this toy example (various leetcode questions and other projects), but I'm happy with the practice that I get
here.

In [19]:
import sqlite3
from sqlite3 import Error

 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

 
def select_all_tasks(conn, query):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = query
    
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)

In [20]:
# Set the database and connection we will use

database = "sqlite_db_pythonsqlite.db"
conn=create_connection(database)

2.6.0


Let's check out what one of the tables looks like!

In [48]:
query = """
SELECT *
FROM Bookings
LIMIT 10
"""

with conn:
    select_all_tasks(conn, query)

(0, 3, 1, '2012-07-03 11:00:00', 2)
(1, 4, 1, '2012-07-03 08:00:00', 2)
(2, 6, 0, '2012-07-03 18:00:00', 2)
(3, 7, 1, '2012-07-03 19:00:00', 2)
(4, 8, 1, '2012-07-03 10:00:00', 1)
(5, 8, 1, '2012-07-03 15:00:00', 1)
(6, 0, 2, '2012-07-04 09:00:00', 3)
(7, 0, 2, '2012-07-04 15:00:00', 3)
(8, 4, 3, '2012-07-04 13:30:00', 2)
(9, 4, 0, '2012-07-04 15:00:00', 2)


Some of the facilities charge a fee to members, but some do not.
Let's see which ones do.

In [22]:
query = """
SELECT name 
FROM Facilities 
WHERE membercost > 0
         """
with conn:
    select_all_tasks(conn, query)

('Tennis Court 1',)
('Tennis Court 2',)
('Massage Room 1',)
('Massage Room 2',)
('Squash Court',)


How many facilities are free for members?

In [26]:
query= """
SELECT COUNT(*)
FROM Facilities
WHERE membercost = 0
"""
with conn:
    select_all_tasks(conn, query)

(4,)


Which facilities charge a fee to members, where the fee is less than 20% of the monthly maintenance? 

In [27]:
query= """
SELECT facid, 
       name,
       membercost,
       monthlymaintenance
FROM Facilities
WHERE membercost > 0
      AND membercost < 0.2 * monthlymaintenance
"""
with conn:
    select_all_tasks(conn, query)

(0, 'Tennis Court 1', 5, 200)
(1, 'Tennis Court 2', 5, 200)
(4, 'Massage Room 1', 9.9, 3000)
(5, 'Massage Room 2', 9.9, 3000)
(6, 'Squash Court', 3.5, 80)


Details of facilities with ID 1 and 5 without using the OR operator

In [28]:
query= """
SELECT *
FROM Facilities
WHERE facid IN (1,5)
"""
with conn:
    select_all_tasks(conn, query)

(1, 'Tennis Court 2', 5, 25, 8000, 200)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)


List of facilities, with each labelled as 'cheap' or 'expensive', depending 
on if their monthly maintenance cost is more than $100.

In [29]:
query= """
SELECT name,
       monthlymaintenance,
       CASE WHEN monthlymaintenance < 100 THEN 'cheap'
            ELSE 'expensive' END AS cost
FROM Facilities
"""
with conn:
    select_all_tasks(conn, query)

('Tennis Court 1', 200, 'expensive')
('Tennis Court 2', 200, 'expensive')
('Badminton Court', 50, 'cheap')
('Table Tennis', 10, 'cheap')
('Massage Room 1', 3000, 'expensive')
('Massage Room 2', 3000, 'expensive')
('Squash Court', 80, 'cheap')
('Snooker Table', 15, 'cheap')
('Pool Table', 15, 'cheap')


Select the first and last names of latest members

In [30]:
query= """
SELECT firstname, 
       surname
FROM Members
WHERE joindate = (SELECT MAX(joindate)
                  FROM Members)
"""
with conn:
    select_all_tasks(conn, query)

('Darren', 'Smith')


List of all members who have used a tennis court. Including the name of the 
court, and the name of the member formatted as a single column. No duplicated data, and ordered by the member name.

In [39]:
query= """
SELECT surname || ' ' || firstname || ' ' || 
       CASE WHEN facid = 0 THEN 'Tennis Court 1' 
            WHEN facid = 1 THEN 'Tennis Court 2' END AS name_and_court 
FROM Bookings
LEFT JOIN Members
On Bookings.memid = Members.memid
WHERE Bookings.facid IN (0,1)
GROUP BY Members.memid
ORDER BY surname, firstname
"""
with conn:
    select_all_tasks(conn, query)

('Bader Florence Tennis Court 2',)
('Baker Anne Tennis Court 1',)
('Baker Timothy Tennis Court 2',)
('Boothe Tim Tennis Court 2',)
('Butters Gerald Tennis Court 1',)
('Coplin Joan Tennis Court 1',)
('Crumpet Erica Tennis Court 1',)
('Dare Nancy Tennis Court 2',)
('Farrell David Tennis Court 1',)
('Farrell Jemima Tennis Court 2',)
('GUEST GUEST Tennis Court 2',)
('Genting Matthew Tennis Court 1',)
('Hunt John Tennis Court 1',)
('Jones David Tennis Court 2',)
('Jones Douglas Tennis Court 1',)
('Joplette Janice Tennis Court 1',)
('Owen Charles Tennis Court 1',)
('Pinker David Tennis Court 1',)
('Purview Millicent Tennis Court 2',)
('Rownam Tim Tennis Court 2',)
('Rumney Henrietta Tennis Court 2',)
('Sarwin Ramnaresh Tennis Court 2',)
('Smith Darren Tennis Court 2',)
('Smith Jack Tennis Court 1',)
('Smith Tracy Tennis Court 1',)
('Stibbons Ponder Tennis Court 2',)
('Tracy Burton Tennis Court 2',)


List of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30. Included in the output is the name of the facility, the name of the member formatted as a single column, and the cost. Ordered by descending cost, with no subqueries.

In [55]:
query= """
SELECT f.name,
       CASE WHEN m.memid != 0 THEN m.firstname || ' ' || m.surname
            ELSE  m.firstname END AS name,
        CASE WHEN m.memid != 0 THEN slots*membercost 
            ELSE slots*guestcost END AS cost
FROM Bookings AS b
LEFT JOIN Facilities AS f
ON b.facid = f.facid
LEFT JOIN Members AS m
ON b.memid = m.memid
WHERE strftime('%Y%m%d', starttime)='20120914' AND
      cost>30
ORDER BY cost DESC
"""
with conn:
    select_all_tasks(conn, query)

('Massage Room 2', 'GUEST', 320)
('Massage Room 1', 'GUEST', 160)
('Massage Room 1', 'GUEST', 160)
('Massage Room 1', 'GUEST', 160)
('Tennis Court 2', 'GUEST', 150)
('Tennis Court 1', 'GUEST', 75)
('Tennis Court 1', 'GUEST', 75)
('Tennis Court 2', 'GUEST', 75)
('Squash Court', 'GUEST', 70.0)
('Massage Room 1', 'Jemima Farrell', 39.6)
('Squash Court', 'GUEST', 35.0)
('Squash Court', 'GUEST', 35.0)


Same as before, but using a subquery!

In [56]:
query= """
SELECT facility_name,
       name,
       cost
FROM (SELECT name AS facility_name,
      CASE WHEN m.memid != 0 THEN surname || ' ' || firstname 
      ELSE "GUEST" END AS name,b.facid,
      b.starttime,
      b.memid,
      CASE WHEN m.memid != 0 THEN slots*membercost 
      ELSE slots*guestcost END AS cost
      FROM Bookings AS b
      INNER JOIN Members AS m
      ON b.memid = m.memid
      INNER JOIN Facilities AS f
      ON b.facid = f.facid) AS b
WHERE strftime('%Y%m%d', starttime)='20120914' AND
      cost>30
ORDER BY cost DESC
"""
with conn:
    select_all_tasks(conn, query)

('Massage Room 2', 'GUEST', 320)
('Massage Room 1', 'GUEST', 160)
('Massage Room 1', 'GUEST', 160)
('Massage Room 1', 'GUEST', 160)
('Tennis Court 2', 'GUEST', 150)
('Tennis Court 1', 'GUEST', 75)
('Tennis Court 1', 'GUEST', 75)
('Tennis Court 2', 'GUEST', 75)
('Squash Court', 'GUEST', 70.0)
('Massage Room 1', 'Farrell Jemima', 39.6)
('Squash Court', 'GUEST', 35.0)
('Squash Court', 'GUEST', 35.0)


List of facilities with a toal revenue less than 1000.

In [58]:
query= """
SELECT cost,
       name
FROM (SELECT ROUND(SUM(CASE WHEN b.memid != 0 THEN slots*membercost
                             ELSE slots*guestcost END),2) AS cost,
             name
      FROM Bookings AS b
      LEFT JOIN Facilities AS f
      ON b.facid = f.facid
      GROUP BY f.facid) AS f
WHERE cost < 1000
GROUP BY name
ORDER BY cost DESC
"""
with conn:
    select_all_tasks(conn, query)

(270.0, 'Pool Table')
(240.0, 'Snooker Table')
(180.0, 'Table Tennis')


Report of members and who recommended them in alphabetic surnam, firstname order

In [59]:
query= """
SELECT m1.surname AS surname,
       m1.firstname AS firstname,
       m2.surname AS recommender_surname,
       m2.firstname AS recommender_firstname
FROM Members AS m1
LEFT JOIN Members AS m2
ON m1.recommendedby = m2.memid
ORDER BY surname, firstname
"""
with conn:
    select_all_tasks(conn, query)

('Bader', 'Florence', 'Stibbons', 'Ponder')
('Baker', 'Anne', 'Stibbons', 'Ponder')
('Baker', 'Timothy', 'Farrell', 'Jemima')
('Boothe', 'Tim', 'Rownam', 'Tim')
('Butters', 'Gerald', 'Smith', 'Darren')
('Coplin', 'Joan', 'Baker', 'Timothy')
('Crumpet', 'Erica', 'Smith', 'Tracy')
('Dare', 'Nancy', 'Joplette', 'Janice')
('Farrell', 'David', None, None)
('Farrell', 'Jemima', None, None)
('GUEST', 'GUEST', None, None)
('Genting', 'Matthew', 'Butters', 'Gerald')
('Hunt', 'John', 'Purview', 'Millicent')
('Jones', 'David', 'Joplette', 'Janice')
('Jones', 'Douglas', 'Jones', 'David')
('Joplette', 'Janice', 'Smith', 'Darren')
('Mackenzie', 'Anna', 'Smith', 'Darren')
('Owen', 'Charles', 'Smith', 'Darren')
('Pinker', 'David', 'Farrell', 'Jemima')
('Purview', 'Millicent', 'Smith', 'Tracy')
('Rownam', 'Tim', None, None)
('Rumney', 'Henrietta', 'Genting', 'Matthew')
('Sarwin', 'Ramnaresh', 'Bader', 'Florence')
('Smith', 'Darren', None, None)
('Smith', 'Darren', None, None)
('Smith', 'Jack', 'Smith',

List of facilities with their usage by member, but not guest

In [62]:
query= """
SELECT COUNT(CASE WHEN b.memid != 0 AND b.memid IS NOT NULL THEN 1
                  ELSE 0 END),
       f.name
FROM facilities AS f
LEFT JOIN Bookings AS b
ON b.facid = f.facid
GROUP BY b.facid
"""
with conn:
    select_all_tasks(conn, query)

(408, 'Tennis Court 1')
(389, 'Tennis Court 2')
(383, 'Badminton Court')
(403, 'Table Tennis')
(629, 'Massage Room 1')
(111, 'Massage Room 2')
(440, 'Squash Court')
(444, 'Snooker Table')
(836, 'Pool Table')


List of facilities with their usage by month, but not guest

In [69]:
query= """
SELECT COUNT(CASE WHEN b.memid != 0 AND b.memid IS NOT NULL THEN 1
                  ELSE 0 END),
       strftime('%m', starttime) AS month,
       f.name
FROM facilities AS f
LEFT OUTER JOIN Bookings AS b
ON b.facid = f.facid
GROUP BY month, f.name
"""
with conn:
    select_all_tasks(conn, query)

(56, '07', 'Badminton Court')
(123, '07', 'Massage Room 1')
(12, '07', 'Massage Room 2')
(110, '07', 'Pool Table')
(75, '07', 'Snooker Table')
(75, '07', 'Squash Court')
(51, '07', 'Table Tennis')
(88, '07', 'Tennis Court 1')
(68, '07', 'Tennis Court 2')
(146, '08', 'Badminton Court')
(224, '08', 'Massage Room 1')
(40, '08', 'Massage Room 2')
(291, '08', 'Pool Table')
(159, '08', 'Snooker Table')
(170, '08', 'Squash Court')
(147, '08', 'Table Tennis')
(146, '08', 'Tennis Court 1')
(149, '08', 'Tennis Court 2')
(181, '09', 'Badminton Court')
(282, '09', 'Massage Room 1')
(59, '09', 'Massage Room 2')
(435, '09', 'Pool Table')
(210, '09', 'Snooker Table')
(195, '09', 'Squash Court')
(205, '09', 'Table Tennis')
(174, '09', 'Tennis Court 1')
(172, '09', 'Tennis Court 2')


That marks the end of the queries! If you got to the bottom, I am extremely impressed at your ability to scroll down to the bottom of pages without reading most of the contents!